<span style="display:block;text-align:center;margin-right:105px"><img src="../../media/logos/logo-vertical.png" width="200"/></span>

# Section 6: Advanced Simulation Methods Exercises

---

## Exercise 1: Monte Carlo Method

Given the following simulation configuration, how many runs would you expect cadCAD to create?

```python
system_params = {
    'parameter_1': [i * 0.1 for i in range(10)]
}

SIMULATION_TIMESTEPS = 500
MONTE_CARLO_RUNS = 10

sim_config = config_sim({
    'N': MONTE_CARLO_RUNS,
    'T': range(SIMULATION_TIMESTEPS),
    'M': system_params
})

experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config,
)
```

*Answer here in the markdown cell:*

## Exercise 2: Parameter Sweeps

Given the following two parameter lists, configure a parameter sweep for all possible combinations of the two parameters.

```python
parameter_1 = [1, 2]
parameter_2 = [0, 10, 20, 30]
```

In [ ]:
# The following statement is incomplete - fill in the gaps and run the cell

system_params = {
    '': [],
    '': []
}
system_params

## Exercise 3: A/B Testing

Given the following function (a failure rate calculation for a manufacturing process), and the two models `a` and `b`, fill in the gaps and run an A/B test.

In [ ]:
import pandas as pd
import plotly
import plotly.express as px

pd.options.plotting.backend = "plotly"

from cadCAD.configuration.utils import config_sim
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor
from cadCAD.configuration import Experiment

In [ ]:
# Run this cell: failure rate calculation

def failure_rate(speed, limit=100, alpha=0):
    # If the speed is greater than the limit, return a 100% failure rate
    if speed >= limit:
        return 1
    # Else if the speed is less than 1, return a 0% failure rate to bound the system
    elif speed <= 1:
        return 0
    # Else calculate a sigmoid like function of the failure rate
    else:
        return 2 * limit / (2 * limit + (speed / (2 * limit - speed))**(-alpha))

In [ ]:
# Run this cell: model A initial state and parameterization

initial_state_a = {
    'failure_rate': 0
}

alpha_start = 0
alpha_end = 5

system_params_a = {
    'alpha': list(range(alpha_start, alpha_end + 1)),
    'limit': [100]
}

In [ ]:
# Run this cell: model B initial state and parameterization

initial_state_b = {
    'failure_rate': 0
}

alpha_start = 5
alpha_end = 10

system_params_b = {
    'alpha': list(range(alpha_start, alpha_end + 1)),
    'limit': [100]
}

In [ ]:
# Run this cell: shared policy and state update functions

def p_assembly_process(params, substep, state_history, previous_state):
    speed = previous_state['timestep']
    limit = params['limit']
    alpha = params['alpha']
    
    delta_failure_rate = failure_rate(speed, limit, alpha)
    return {'delta_failure_rate': delta_failure_rate}
    
def s_failure_rate(params, substep, state_history, previous_state, policy_input):
    delta_failure_rate = policy_input['delta_failure_rate']
    return 'failure_rate', delta_failure_rate

In [ ]:
# Run this cell: shared partial state update blocks

partial_state_update_blocks = [
    {
        'policies': {
            'manufacture': p_assembly_process
        },
        'variables': {
            'failure_rate': s_failure_rate
        }
    }
]

In [ ]:
# Run this cell: shared configuration

SIMULATION_TIMESTEPS = 100
MONTE_CARLO_RUNS = 1

experiment = Experiment()

from cadCAD import configs
del configs[:] # Clear any prior configs

Update the following two cells:

In [ ]:
# The following statements are incomplete - fill in the gaps and run the cell

# Model A simulation configuration
sim_config = config_sim({})

experiment.append_configs()

In [ ]:
# The following statements are incomplete - fill in the gaps and run the cell

# Model B simulation configuration
sim_config = config_sim({})

experiment.append_configs()

In [ ]:
# Run this cell

exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()
simulation_result = pd.DataFrame(raw_result)

In [ ]:
# Run this cell

fig = px.line(
    simulation_result,
    x='timestep',
    y=['failure_rate'],
    facet_row='simulation',
    facet_col='run',
    height=800,
    template='seaborn',
)

fig.show()

<br/><br/><br/>
# Well done!
<br/><br/><br/><br/>